[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

如果要訓練這份資料集會需要安裝talib套件，請參考[這裡](https://www.lfd.uci.edu/~gohlke/pythonlibs/#ta-lib)下載對應的版本，並使用pip安裝。

In [ ]:
# !pip install ccxt

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [1]:
from ccxt import binance
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from torch.nn import functional as F
from talib import abstract
import pandas as pd
import numpy as np
import torch
import os
import sys

# 環境整理

In [2]:
ls_dir = os.listdir(path="..")
if "data" not in ls_dir:
    os.mkdir(path="../data")
if "model" not in ls_dir:
    os.mkdir(path="../model")
if "data" in ls_dir:
    ls_dir = os.listdir(path="../data")
    #remove all files in data folder
    for file in ls_dir:os.remove(path="../data/"+file)

# 下載資料集

In [3]:
# Binance BTC/USDT 1h candles from 2020-01-01 to 2021-01-01

binance = binance()
symbol = 'BTC/USDT'
timeframe = '1h'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 3600000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

# 讀取資料集

In [ ]:
#如果有下载好的數據，可以直接讀取
data_file = '../data/btc_usdt_1h.csv' #-> 可以自行更換
df = pd.read_csv(data_file)

# 數據處理

- 計算RSI
- 計算MACD
- 計算OBV
- 計算CCI
- 改成變化百分比 -> 標準化

關於技術指標的說明可以參考[這裡](https://www.investopedia.com/terms/t/technicalindicator.asp)，或是google。

In [4]:
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['MACD'] = abstract.MACD(df, fastperiod=12, slowperiod=26, signalperiod=9)['macd'] #只取MACD
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()
#由於RSI MACD OBV CCI 他們已經是標準化的，所以不需要再標準化

# 設定買賣點

將買賣點分為下跌、上漲、不動，並將數據轉成one-hot編碼。

In [5]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()


In [6]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,CLOSE_percent,UP,DOWN
33,2019-10-02 09:00:00,8237.59,8267.26,8220.46,8234.67,730.375055,48.723435,-36.978752,-5388.066483,-32.078733,79.716399,16.445918,43.517312,-0.000397,1.0,0.0
34,2019-10-02 10:00:00,8235.00,8258.78,8211.05,8255.93,721.600120,51.222502,-33.420223,-4666.466363,-22.331369,77.431656,15.656964,40.954942,0.002582,1.0,0.0
35,2019-10-02 11:00:00,8255.47,8289.00,8225.53,8269.04,912.918303,52.751719,-29.205532,-3753.548060,21.716060,76.434395,15.521321,47.604680,0.001588,0.0,1.0
36,2019-10-02 12:00:00,8268.67,8300.00,8208.00,8229.10,1510.638503,47.831816,-28.756695,-5264.186563,0.716845,77.546224,14.970377,45.298104,-0.004830,0.0,1.0
37,2019-10-02 13:00:00,8230.40,8261.04,8200.00,8216.80,1317.184129,46.396694,-29.058526,-6581.370692,-38.372502,76.367208,14.265805,44.030923,-0.001495,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28453,2022-12-31 00:00:00,16607.48,16616.37,16578.00,16580.32,4264.258480,53.104081,2.759274,-393124.472793,89.212926,58.593323,26.989753,48.874053,-0.001635,0.0,1.0
28454,2022-12-31 01:00:00,16580.32,16583.69,16562.36,16568.24,4007.775890,51.354857,3.169750,-397132.248683,50.044160,55.931657,25.522369,49.121993,-0.000729,1.0,0.0
28455,2022-12-31 02:00:00,16568.23,16579.79,16560.47,16575.94,3569.361150,52.430467,4.069471,-393562.887533,44.708698,53.316539,24.112844,54.624178,0.000465,0.0,1.0
28456,2022-12-31 03:00:00,16576.18,16577.38,16550.01,16552.46,3814.652160,48.881119,2.854956,-397377.539693,14.442228,51.463215,22.535743,55.957329,-0.001417,0.0,1.0


In [7]:
#正規化
df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
df['OBV'] = (df['OBV'] - df['OBV'].min()) / (df['OBV'].max() - df['OBV'].min())
df['CCI'] = (df['CCI'] - df['CCI'].min()) / (df['CCI'].max() - df['CCI'].min())
df['ATR'] = (df['ATR'] - df['ATR'].min()) / (df['ATR'].max() - df['ATR'].min())
df['ADX'] = (df['ADX'] - df['ADX'].min()) / (df['ADX'].max() - df['ADX'].min())
df['MFI'] = (df['MFI'] - df['MFI'].min()) / (df['MFI'].max() - df['MFI'].min())
df['open'] = (df['open'] - df['open'].min()) / (df['open'].max() - df['open'].min())
df['high'] = (df['high'] - df['high'].min()) / (df['high'].max() - df['high'].min())
df['low'] = (df['low'] - df['low'].min()) / (df['low'].max() - df['low'].min())
df['close'] = (df['close'] - df['close'].min()) / (df['close'].max() - df['close'].min())
df['volume'] = (df['volume'] - df['volume'].min()) / (df['volume'].max() - df['volume'].min())

C:\Users\李軒豪\AppData\Local\Temp\ipykernel_15624\1144569331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = (df['RSI'] - df['RSI'].min()) / (df['RSI'].max() - df['RSI'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_15624\1144569331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD'] = (df['MACD'] - df['MACD'].min()) / (df['MACD'].max() - df['MACD'].min())
C:\Users\李軒豪\AppData\Local\Temp\ipykernel_15624\1144569331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [8]:
df.drop(['CLOSE_percent'], axis=1, inplace=True)

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
df

,time,open,high,low,close,volume,RSI,MACD,OBV,CCI,ATR,ADX,MFI,UP,DOWN
33,2019-10-02 09:00:00,0.063661,0.056799,0.068631,0.063625,0.005323,0.462730,0.547051,0.349256,0.465913,0.026936,0.131840,0.435173,1.0,0.0
34,2019-10-02 10:00:00,0.063621,0.056667,0.068486,0.063955,0.005259,0.491218,0.547889,0.349723,0.476396,0.025946,0.121307,0.409549,1.0,0.0
35,2019-10-02 11:00:00,0.063938,0.057136,0.068710,0.064158,0.006654,0.508651,0.548881,0.350314,0.523771,0.025513,0.119497,0.476047,0.0,1.0
36,2019-10-02 12:00:00,0.064142,0.057307,0.068439,0.063539,0.011010,0.452566,0.548987,0.349336,0.501186,0.025995,0.112142,0.452981,0.0,1.0
37,2019-10-02 13:00:00,0.063549,0.056702,0.068315,0.063348,0.009600,0.436206,0.548916,0.348484,0.459144,0.025484,0.102736,0.440309,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28453,2022-12-31 00:00:00,0.193419,0.186463,0.197867,0.193009,0.031079,0.512667,0.556406,0.098259,0.596367,0.017779,0.272597,0.488741,0.0,1.0
28454,2022-12-31 01:00:00,0.192998,0.185956,0.197625,0.192822,0.029210,0.492727,0.556502,0.095665,0.554239,0.016625,0.253007,0.491220,1.0,0.0
28455,2022-12-31 02:00:00,0.192810,0.185895,0.197596,0.192941,0.026014,0.504989,0.556714,0.097975,0.548501,0.015492,0.234191,0.546242,0.0,1.0
28456,2022-12-31 03:00:00,0.192933,0.185858,0.197434,0.192577,0.027802,0.464528,0.556428,0.095506,0.515948,0.014688,0.213137,0.559573,0.0,1.0


In [10]:
df['UP'].value_counts()

1.0    14385
0.0    14040
Name: UP, dtype: int64

In [11]:
df['DOWN'].value_counts()

0.0    14388
1.0    14037
Name: DOWN, dtype: int64

看起來數據蠻平衡的

# 儲存資料

In [12]:
df.to_csv(file_name, index=False)

# 分割成X、y

In [13]:

X,y = list(),list()
ref_bar = 10

for i in range(len(df)-ref_bar):
    #df.iloc.values 會回傳一個numpy array
    X.append(df.iloc[i:i+ref_bar, 1:13].values) # i to i+ref_bar-1
    y.append(df.iloc[i+ref_bar-1, 13:].values) # i+ref_bar-1



In [14]:
X[0]#這裡會出現10個array，每個array裡面有12個數字，分別是open high low close volume RSI MACD OBV CCI ATR ADX MFI

array([[0.06366066, 0.05679857, 0.06863143, 0.06362536, 0.00532315,
        0.46273012, 0.54705143, 0.34925629, 0.46591279, 0.02693599,
        0.13183968, 0.43517312],
       [0.0636205 , 0.05666687, 0.06848592, 0.06395496, 0.0052592 ,
        0.49121834, 0.54788909, 0.34972341, 0.47639645, 0.02594554,
        0.12130741, 0.40954942],
       [0.06393785, 0.0571362 , 0.06870983, 0.06415821, 0.00665357,
        0.50865072, 0.54888121, 0.35031437, 0.52377115, 0.02551323,
        0.11949661, 0.4760468 ],
       [0.06414249, 0.05730704, 0.06843876, 0.06353901, 0.01100991,
        0.45256607, 0.54898686, 0.34933648, 0.50118569, 0.02599521,
        0.11214169, 0.45298104],
       [0.06354919, 0.05670197, 0.06831505, 0.06334832, 0.00959996,
        0.43620634, 0.54891581, 0.34848381, 0.4591436 , 0.0254841 ,
        0.10273587, 0.44030923],
       [0.0633382 , 0.05653052, 0.06811851, 0.0636528 , 0.00966987,
        0.46618582, 0.5493069 , 0.34934269, 0.46610558, 0.02506121,
        0.08988118,

In [15]:
y[0]

array([1.0, 0.0], dtype=object)

In [16]:
X = np.array(X)
X = torch.tensor(X, dtype=torch.float32)

In [17]:
y = np.array(y,dtype=np.float32)
y = torch.tensor(y, dtype=torch.float32)

In [18]:
y

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [0., 1.]])

In [19]:
print(f"X shape: {X.shape} , y shape: {y.shape}")

X shape: torch.Size([28415, 10, 12]) , y shape: torch.Size([28415, 2])


In [20]:
X[0]

tensor([[0.0637, 0.0568, 0.0686, 0.0636, 0.0053, 0.4627, 0.5471, 0.3493, 0.4659,
         0.0269, 0.1318, 0.4352],
        [0.0636, 0.0567, 0.0685, 0.0640, 0.0053, 0.4912, 0.5479, 0.3497, 0.4764,
         0.0259, 0.1213, 0.4095],
        [0.0639, 0.0571, 0.0687, 0.0642, 0.0067, 0.5087, 0.5489, 0.3503, 0.5238,
         0.0255, 0.1195, 0.4760],
        [0.0641, 0.0573, 0.0684, 0.0635, 0.0110, 0.4526, 0.5490, 0.3493, 0.5012,
         0.0260, 0.1121, 0.4530],
        [0.0635, 0.0567, 0.0683, 0.0633, 0.0096, 0.4362, 0.5489, 0.3485, 0.4591,
         0.0255, 0.1027, 0.4403],
        [0.0633, 0.0565, 0.0681, 0.0637, 0.0097, 0.4662, 0.5493, 0.3493, 0.4661,
         0.0251, 0.0899, 0.4219],
        [0.0637, 0.0565, 0.0685, 0.0637, 0.0069, 0.4717, 0.5498, 0.3500, 0.5146,
         0.0239, 0.0779, 0.4315],
        [0.0637, 0.0569, 0.0684, 0.0634, 0.0097, 0.4394, 0.5498, 0.3491, 0.5221,
         0.0236, 0.0738, 0.4196],
        [0.0634, 0.0564, 0.0680, 0.0636, 0.0097, 0.4588, 0.5501, 0.3500, 0.4577,

In [21]:
y[0]

tensor([1., 0.])

# 建立資料集類別

- 要繼承torch.utils.data.Dataset
- 要實作`__len__`、`__getitem__`
- 後面要用DataLoader取用

In [22]:
#用sklearn來分成train val test
# train:val:test = 6:3:1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.75, random_state=42)

In [23]:
print(f"X_train shape: {X_train.shape} , y_train shape: {y_train.shape}")

X_train shape: torch.Size([17049, 10, 12]) , y_train shape: torch.Size([17049, 2])


In [24]:
print(f"X_test shape: {X_test.shape} , y_test shape: {y_test.shape}")

X_test shape: torch.Size([8525, 10, 12]) , y_test shape: torch.Size([8525, 2])


In [25]:
print(f"X_val shape: {X_val.shape} , y_val shape: {y_val.shape}")

X_val shape: torch.Size([2841, 10, 12]) , y_val shape: torch.Size([2841, 2])


In [26]:
class TrainDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [27]:
class ValDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [28]:
class TestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [29]:
train_dataset = TrainDataset(X_train, y_train)
val_dataset = ValDataset(X_val, y_val)
test_dataset = TestDataset(X_test, y_test)

# 建立模型

- 要繼承torch.nn.Module
- 可能要多建立不同的模型，到時候看結果再調整 -> 先讓數據流得通，再去看成績做調整。

- ver1 -> CNN+MLP
- ver2 -> CNN+LSTM+MLP
- ver3 -> CNN+GRU+MLP
- ver4 -> CNN+LSTM+GRU+MLP
- ver5 -> CNN+GRU+MLP+Dropout

In [30]:
class SelectItem(torch.nn.Module):#這是用來取出多個輸出其中一個的輸出，如果不用sequential的話，就可以不用這個
    def __init__(self, item_index):
        super(SelectItem, self).__init__()
        self._name = 'selectitem'
        self.item_index = item_index

    def forward(self, inputs):
        return inputs[self.item_index]

In [82]:
class crypto_classfier_ver1(nn.Module): #CNN+MLP
    def __init__(self):
        super(crypto_classfier_ver1, self).__init__()
        self.name = "CCV-1"
        self.spare_layer = nn.Sequential(
            torch.nn.Conv1d(10, 4, 3, stride=4, padding=1),
            torch.nn.ReLU())
        self.net = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 512),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1024), 
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 2),
            torch.nn.ReLU())
        self.fusion = nn.Sequential(
            torch.nn.Linear(6, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0))

    def forward(self, x):
        x = self.spare_layer(x)
        x = x.T
        x1 = self.net(x[0])
        x2 = self.net(x[1])
        x3 = self.net(x[2])
        x = torch.cat((x1,x2,x3),0)
        x = self.fusion(x)
        return x

In [91]:
class crypto_classfier_ver2(nn.Module): #CNN+LSTM+MLP
    def __init__(self):
        super(crypto_classfier_ver2, self).__init__()
        self.name = "CCV-2"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.Softmax(dim=0))
    def forward(self, x):
        x = self.net(x)
        return x

In [106]:
class crypto_classfier_ver3(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver3, self).__init__()
        self.name = "CCV-3"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(64,2),
            torch.nn.ELU()
            )
    def forward(self, x):
        x = self.net(x)
        # print(f"=>{F.softmax(x, dim=0)}")
        x = F.softmax(x, dim=0)
        return x

In [339]:
class crypto_classfier_ver4(nn.Module):#CNN+LSTM+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver4, self).__init__()
        self.name = "CCV-4"
        self.batch_size = 1
        self.spare_layer = torch.nn.Conv1d(10, 2, 3, stride=3, padding=1)
        self.gru = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.lstm = nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 128),
            torch.nn.ELU(),
            torch.nn.LSTM(128, 256, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(256, 512, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(512, 256, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(256, 128, 25),
            SelectItem(0),
            torch.nn.Dropout(0.2),
            torch.nn.LSTM(128, 64, 25),
            SelectItem(0),
            torch.nn.Linear(64, 2))
        self.fusion = nn.Sequential(
            SelectItem(0),
            SelectItem(0),
            torch.nn.Linear(4, 16),
            torch.nn.ELU(),
            torch.nn.Linear(16, 2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.spare_layer(x)
        x1 , x2 = x[0], x[1]
        x1 = x1.view(1,1,4)
        x2 = x2.view(1,1,4)
        out1 = self.gru(x1)
        out2 = self.lstm(x2)
        x = torch.cat((out1,out2),2)
        x = self.fusion(x)
        return x


In [352]:
class CNNwithDropout(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNNwithDropout, self).__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(1 , 128),
            nn.Linear(128, out_channels)
        )
        self.output_layer = nn.Sequential(
            nn.Conv1d(64, 1, kernel_size=3, padding=1),
            SelectItem(0),
            nn.Linear(2,16),
            nn.Linear(16,64),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(64,128),
            nn.Linear(128,2),
            nn.ELU(),
            nn.Softmax(dim=0)
        )

    def forward(self, x):
        return self.output_layer(self.net(x))

In [281]:
class crypto_classfier_ver5(nn.Module):#CNN+GRU+MLP
    def __init__(self):
        super(crypto_classfier_ver5, self).__init__()
        self.name = "CCV-5"
        self.net = nn.Sequential(
            torch.nn.Conv1d(10, 20, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(20, 40, 3, stride=1, padding=1),
            torch.nn.ELU(),
            torch.nn.MaxPool1d(1, stride=1),
            torch.nn.Conv1d(40, 1, 3, stride=1, padding=1),
            torch.nn.Linear(12,64),
            torch.nn.Linear(64,128),
            torch.nn.GRU(128, 64, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(64, 32, 25),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.GRU(32, 16, 25),
            SelectItem(0),
            SelectItem(0),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(16,2),
            torch.nn.Softmax(dim=0)
            )
    def forward(self, x):
        x = self.net(x)
        return x

In [33]:
lab_tensor = train_dataset[0][0]
lab_ans_tensor = train_dataset[0][1]
print(lab_tensor.shape)
print(lab_ans_tensor.shape)
print(lab_tensor)
print(lab_ans_tensor)

torch.Size([10, 12])
torch.Size([2])
tensor([[0.2702, 0.2641, 0.2746, 0.2708, 0.0322, 0.4950, 0.5557, 0.5632, 0.5154,
         0.0481, 0.0735, 0.4561],
        [0.2708, 0.2640, 0.2747, 0.2701, 0.0291, 0.4625, 0.5553, 0.5606, 0.5015,
         0.0459, 0.0681, 0.4930],
        [0.2701, 0.2633, 0.2741, 0.2698, 0.0300, 0.4477, 0.5545, 0.5579, 0.4507,
         0.0438, 0.0690, 0.4103],
        [0.2697, 0.2628, 0.2715, 0.2678, 0.0702, 0.3701, 0.5516, 0.5517, 0.2907,
         0.0460, 0.0896, 0.4142],
        [0.2678, 0.2618, 0.2720, 0.2681, 0.0395, 0.3869, 0.5498, 0.5552, 0.3077,
         0.0452, 0.1087, 0.4338],
        [0.2681, 0.2616, 0.2713, 0.2685, 0.0562, 0.4097, 0.5489, 0.5602, 0.3436,
         0.0454, 0.1306, 0.3320],
        [0.2685, 0.2618, 0.2723, 0.2680, 0.0394, 0.3856, 0.5476, 0.5567, 0.3889,
         0.0440, 0.1485, 0.3319],
        [0.2680, 0.2616, 0.2725, 0.2683, 0.0364, 0.4059, 0.5471, 0.5599, 0.4150,
         0.0419, 0.1651, 0.3273],
        [0.2683, 0.2615, 0.2716, 0.2677, 0.

In [349]:
lab_models = crypto_classfier_ver4()
lab_out = lab_models(lab_tensor)
print(lab_out)

tensor([0.6358, 0.3642], grad_fn=<SoftmaxBackward0>)


In [312]:
lab_target = torch.tensor([0.,1.])
lab_test_ansers = [torch.tensor([1-(i*0.1),i*0.1]) for i in range(1,10)]

In [317]:
for i in lab_test_ansers:
    print(f"BCE loss:{F.binary_cross_entropy(i, lab_target):.4f} ans:{i}")
    print(f"CE loss:{F.cross_entropy(i.view(1,2), torch.tensor([1])):.4f} ans:{i}")
    print(f"MSE loss:{F.mse_loss(i, lab_target):.4f} ans:{i}")
    print("=====================================")

BCE loss:2.3026 ans:tensor([0.9000, 0.1000])
CE loss:1.1711 ans:tensor([0.9000, 0.1000])
MSE loss:0.8100 ans:tensor([0.9000, 0.1000])
NLL loss:-0.1000 ans:tensor([0.9000, 0.1000])
BCE loss:1.6094 ans:tensor([0.8000, 0.2000])
CE loss:1.0375 ans:tensor([0.8000, 0.2000])
MSE loss:0.6400 ans:tensor([0.8000, 0.2000])
NLL loss:-0.2000 ans:tensor([0.8000, 0.2000])
BCE loss:1.2040 ans:tensor([0.7000, 0.3000])
CE loss:0.9130 ans:tensor([0.7000, 0.3000])
MSE loss:0.4900 ans:tensor([0.7000, 0.3000])
NLL loss:-0.3000 ans:tensor([0.7000, 0.3000])
BCE loss:0.9163 ans:tensor([0.6000, 0.4000])
CE loss:0.7981 ans:tensor([0.6000, 0.4000])
MSE loss:0.3600 ans:tensor([0.6000, 0.4000])
NLL loss:-0.4000 ans:tensor([0.6000, 0.4000])
BCE loss:0.6931 ans:tensor([0.5000, 0.5000])
CE loss:0.6931 ans:tensor([0.5000, 0.5000])
MSE loss:0.2500 ans:tensor([0.5000, 0.5000])
NLL loss:-0.5000 ans:tensor([0.5000, 0.5000])
BCE loss:0.5108 ans:tensor([0.4000, 0.6000])
CE loss:0.5981 ans:tensor([0.4000, 0.6000])
MSE loss:0.

In [354]:
lab_models = [crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4(), crypto_classfier_ver5(), CNNwithDropout(10,2)]
out_data = list(map(lambda x: x(lab_tensor), lab_models))

for t in out_data:
    print(t)

tensor([0.4837, 0.5163], grad_fn=<SoftmaxBackward0>)
tensor([0.5069, 0.4931], grad_fn=<SoftmaxBackward0>)
tensor([0.4360, 0.5640], grad_fn=<SoftmaxBackward0>)
tensor([0.4983, 0.5017], grad_fn=<SoftmaxBackward0>)
tensor([0.4535, 0.5465], grad_fn=<SoftmaxBackward0>)
tensor([0.5649, 0.4351], grad_fn=<SoftmaxBackward0>)


## Note

1) model
    - `__init__`: define [layers](https://pytorch.org/docs/stable/nn.html)
    - forward: forward pass -> compute prediction
2) loss and optimizer
    - lr: learning rate [default=0.001]
    - momentum: momentum for optimizer [default=0.9]
    - criterion: loss function [in torch.nn] eg.nn.BCELoss()
    - optimizer: optimizer [in torch.optim] eg.torch.optim.SGD()
        - eg. optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
3) training loop
    - forward pass: compute prediction and loss

        ```python
        output = model(data)
        loss = criterion(output, target)
        ```
        
    - backward pass: loss.backward()
    - update weights: optimizer.step()
    - zero the gradients: optimizer.zero_grad()

# TODO

- 寫訓練方法
    - using dataloader
        - batch and epoch
- 寫驗證方法
    - using model.eval()


In [355]:
test_dataloader_temp = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
loss_f = nn.MSELoss()
test_model = crypto_classfier_ver1()
for ind,(x,y) in enumerate(test_dataloader_temp):
    if ind == 50:
        print(x.shape)
        print(y.shape)
        x = x[0]
        y = y
        out= test_model(x)
        out = torch.tensor([1.,0.]) if out[0] > out[1] else torch.tensor([0.,1.])
        loss_val = loss_f(out, y)
        print(loss_val)
        print(out.shape)
        print(out)
        print(y)
        break

torch.Size([1, 10, 12])
torch.Size([1, 2])
tensor(0.)
torch.Size([2])
tensor([1., 0.])
tensor([[1., 0.]])


C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [357]:
#setting loss function and optimizer
loss_functions = [nn.MSELoss(), nn.BCELoss(),nn.CrossEntropyLoss()]
optimizers = [torch.optim.Adam, torch.optim.SGD]
models = [crypto_classfier_ver1(), crypto_classfier_ver2(), crypto_classfier_ver3(), crypto_classfier_ver4()]
history = dict()
lr = 0.0001
epochs = 1
batch_size = 1 #batch size 是一次讀取多少筆資料 我這邊只能設1 因為我們的資料是一筆一筆的
#training
for model in models:
    for loss_function in loss_functions:
        for optimizer in optimizers:
            print("model: ", model.name)
            print("loss_function: ", loss_function)
            print("optimizer: ", optimizer)
            print("lr: ", lr)
            print("epochs: ", epochs)
            print("batch_size: ", batch_size)
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model.to(device)
            print("device: ", device)
            optimizer = optimizer(model.parameters(), lr=lr)
            train_loss = []
            test_loss = []
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            for epoch in range(epochs):
                for i, (x, y) in enumerate(train_loader):
                    model.train()
                    x = x[0]
                    y = y[0]
                    x = x.to(device)
                    y = y.to(device)
                    optimizer.zero_grad()
                    out = model(x)
                    loss = loss_function(out, y)
                    loss.backward()
                    optimizer.step()
                    train_loss.append(loss.item())
                    if i % 1000 == 0:
                        print("Train epoch: ", epoch, "i: ", i, "loss: ", loss.item(), "out: ", out, "y: ", y)
                for i, (x, y) in enumerate(test_loader):
                    model.eval()
                    x = x[0]
                    y = y[0]
                    x = x.to(device)
                    y = y.to(device)
                    out = model(x)
                    loss = loss_function(out, y)
                    test_loss.append([loss.item(), model.parameters()])
                    if i % 1000 == 0:
                        print("epoch: ", epoch, "i: ", i, "loss: ", loss.item())
                if epoch % 10 == 0:
                    print("epoch: ", epoch, "train_loss: ", train_loss[-1], "test_loss: ", test_loss[-1])
            history[model.name + str(loss_function) + str(optimizer)] = [train_loss[-1], test_loss[-1], model.parameters()]



model:  CCV-1
loss_function:  MSELoss()
optimizer:  <class 'torch.optim.adam.Adam'>
lr:  0.0001
epochs:  1
batch_size:  1
device:  cpu
Train epoch:  0 i:  0 loss:  0.1943138688802719 out:  tensor([0.4408, 0.5592], grad_fn=<SoftmaxBackward0>) y:  tensor([0., 1.])
Train epoch:  0 i:  1000 loss:  0.26273655891418457 out:  tensor([0.4874, 0.5126], grad_fn=<SoftmaxBackward0>) y:  tensor([1., 0.])
Train epoch:  0 i:  2000 loss:  0.24515284597873688 out:  tensor([0.4951, 0.5049], grad_fn=<SoftmaxBackward0>) y:  tensor([0., 1.])
Train epoch:  0 i:  3000 loss:  0.24141839146614075 out:  tensor([0.4913, 0.5087], grad_fn=<SoftmaxBackward0>) y:  tensor([0., 1.])
Train epoch:  0 i:  4000 loss:  0.2535020709037781 out:  tensor([0.4965, 0.5035], grad_fn=<SoftmaxBackward0>) y:  tensor([1., 0.])
Train epoch:  0 i:  5000 loss:  0.24800632894039154 out:  tensor([0.4980, 0.5020], grad_fn=<SoftmaxBackward0>) y:  tensor([0., 1.])
Train epoch:  0 i:  6000 loss:  0.24912042915821075 out:  tensor([0.5009, 0.49

KeyboardInterrupt: 